# Parking Lot Occupancy Estimation - Testing

This notebook performs final testing on the test set and generates comprehensive evaluation reports.

**Author:** Aminu Yiwere  
**Date:** November 4, 2025  
**Environment:** Google Colab

---


## 1. Setup and Installation


In [ ]:
# Check if running on Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Running locally")

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install albumentations
!pip install timm
!pip install tqdm
!pip install matplotlib seaborn
!pip install scikit-learn
!pip install plotly
!pip install fpdf  # For PDF report generation

## 2. Import Libraries


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm.auto import tqdm
import time
import json
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models

# Additional imports
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, classification_report,
    roc_curve, auc, roc_auc_score
)
import timm

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

## 3. Configuration


In [ ]:
# Configuration
class Config:
    # Paths
    if IN_COLAB:
        DATA_DIR = '/content/drive/MyDrive/parking_lot_data'
        OUTPUT_DIR = '/content/drive/MyDrive/parking_lot_output'
    else:
        DATA_DIR = './data/processed'
        OUTPUT_DIR = './output'
    
    TEST_DIR = os.path.join(DATA_DIR, 'test')
    CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, 'checkpoints')
    RESULTS_DIR = os.path.join(OUTPUT_DIR, 'test_results')
    
    # Model parameters
    MODEL_NAME = 'resnet50'  # Should match the trained model
    NUM_CLASSES = 2
    
    # Test parameters
    BATCH_SIZE = 32
    IMG_SIZE = 224
    NUM_WORKERS = 2 if IN_COLAB else 4
    
    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
os.makedirs(config.RESULTS_DIR, exist_ok=True)

print(f"Device: {config.DEVICE}")
print(f"Model: {config.MODEL_NAME}")
print(f"Test directory: {config.TEST_DIR}")

## 4. Dataset and DataLoader


In [ ]:
# Custom Dataset Class
class ParkingLotDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = ['occupied', 'vacant']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Load image paths and labels
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            if not os.path.exists(class_dir):
                continue
            
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, img_name)
                    label = self.class_to_idx[class_name]
                    self.samples.append((img_path, label))
        
        print(f"Found {len(self.samples)} images in {data_dir}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply transforms
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, label, img_path

# Test transforms (no augmentation)
def get_test_transforms(img_size=224):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ])

# Create dataset and dataloader
test_dataset = ParkingLotDataset(
    data_dir=config.TEST_DIR,
    transform=get_test_transforms(config.IMG_SIZE)
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    num_workers=config.NUM_WORKERS,
    pin_memory=True
)

print(f"Test samples: {len(test_dataset)}")
print(f"Test batches: {len(test_loader)}")

## 5. Load Trained Model


In [ ]:
def create_model(model_name, num_classes=2):
    """Create model architecture"""
    if model_name == 'resnet50':
        model = models.resnet50(pretrained=False)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    
    elif model_name == 'resnet101':
        model = models.resnet101(pretrained=False)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=False)
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, num_classes)
    
    elif 'efficientnet' in model_name:
        model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    
    else:
        raise ValueError(f"Model {model_name} not supported")
    
    return model

# Load model
model = create_model(config.MODEL_NAME, config.NUM_CLASSES)
checkpoint_path = os.path.join(config.CHECKPOINT_DIR, f'best_model_{config.MODEL_NAME}.pth')

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=config.DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✓ Model loaded from {checkpoint_path}")
    print(f"  Trained for {checkpoint['epoch'] + 1} epochs")
    print(f"  Validation Loss: {checkpoint['val_loss']:.4f}")
    print(f"  Validation Accuracy: {checkpoint['val_acc']:.4f}")
else:
    print(f"⚠ Checkpoint not found at {checkpoint_path}")
    print("Please train the model first using train.ipynb")

model = model.to(config.DEVICE)
model.eval()
print("\nModel ready for testing!")

## 6. Test Function with Inference Time


In [ ]:
def test_model(model, dataloader, device):
    """
    Test model with inference time measurement
    """
    model.eval()
    
    all_preds = []
    all_labels = []
    all_probs = []
    all_paths = []
    inference_times = []
    
    with torch.no_grad():
        for images, labels, paths in tqdm(dataloader, desc='Testing'):
            images = images.to(device)
            labels = labels.to(device)
            
            # Measure inference time
            start_time = time.time()
            outputs = model(images)
            inference_time = time.time() - start_time
            inference_times.append(inference_time / images.size(0))  # Time per image
            
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            # Store predictions
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_paths.extend(paths)
    
    avg_inference_time = np.mean(inference_times)
    fps = 1.0 / avg_inference_time if avg_inference_time > 0 else 0
    
    return (
        np.array(all_preds), 
        np.array(all_labels), 
        np.array(all_probs), 
        all_paths,
        avg_inference_time,
        fps
    )

## 7. Run Test


In [ ]:
# Run test
print("\n" + "="*50)
print("RUNNING FINAL TEST")
print("="*50 + "\n")

predictions, labels, probabilities, image_paths, avg_time, fps = test_model(
    model, test_loader, config.DEVICE
)

print("\n✓ Test complete!")
print(f"  Total samples: {len(predictions)}")
print(f"  Average inference time: {avg_time*1000:.2f} ms/image")
print(f"  Throughput: {fps:.2f} FPS")

## 8. Calculate All Metrics


In [ ]:
# Calculate comprehensive metrics
accuracy = accuracy_score(labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
conf_matrix = confusion_matrix(labels, predictions)

# Per-class metrics
precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
    labels, predictions, average=None
)

# ROC AUC
roc_auc = roc_auc_score(labels, probabilities[:, 1])

# Specificity and Sensitivity
tn, fp, fn, tp = conf_matrix.ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0

print("\n" + "="*60)
print("FINAL TEST RESULTS")
print("="*60)

print(f"\n📊 Overall Performance Metrics:")
print(f"  • Accuracy:    {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  • Precision:   {precision:.4f}")
print(f"  • Recall:      {recall:.4f}")
print(f"  • F1-Score:    {f1:.4f}")
print(f"  • ROC AUC:     {roc_auc:.4f}")
print(f"  • Sensitivity: {sensitivity:.4f}")
print(f"  • Specificity: {specificity:.4f}")

print(f"\n📈 Per-Class Performance:")
print(f"\n  Occupied (Class 0):")
print(f"    Precision: {precision_per_class[0]:.4f}")
print(f"    Recall:    {recall_per_class[0]:.4f}")
print(f"    F1-Score:  {f1_per_class[0]:.4f}")
print(f"    Support:   {support[0]}")

print(f"\n  Vacant (Class 1):")
print(f"    Precision: {precision_per_class[1]:.4f}")
print(f"    Recall:    {recall_per_class[1]:.4f}")
print(f"    F1-Score:  {f1_per_class[1]:.4f}")
print(f"    Support:   {support[1]}")

print(f"\n⚡ Performance:")
print(f"  • Inference Time: {avg_time*1000:.2f} ms/image")
print(f"  • Throughput:     {fps:.2f} FPS")

print("\n" + "="*60)

## 9. Confusion Matrix


In [ ]:
def plot_confusion_matrix(conf_matrix, class_names=['Occupied', 'Vacant']):
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Raw counts
    sns.heatmap(
        conf_matrix,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names,
        square=True,
        ax=axes[0],
        cbar_kws={'label': 'Count'}
    )
    axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, pad=15)
    axes[0].set_ylabel('True Label', fontsize=12)
    axes[0].set_xlabel('Predicted Label', fontsize=12)
    
    # Normalized
    conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
    sns.heatmap(
        conf_matrix_normalized,
        annot=True,
        fmt='.2%',
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names,
        square=True,
        ax=axes[1],
        cbar_kws={'label': 'Percentage'}
    )
    axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, pad=15)
    axes[1].set_ylabel('True Label', fontsize=12)
    axes[1].set_xlabel('Predicted Label', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
    plt.show()

plot_confusion_matrix(conf_matrix)

## 10. ROC Curve


In [ ]:
def plot_roc_curve(labels, probabilities):
    fpr, tpr, thresholds = roc_curve(labels, probabilities[:, 1])
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=3, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=13)
    plt.ylabel('True Positive Rate', fontsize=13)
    plt.title('Receiver Operating Characteristic (ROC) Curve - Test Set', fontsize=15, pad=20)
    plt.legend(loc='lower right', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'roc_curve.png'), dpi=300, bbox_inches='tight')
    plt.show()

plot_roc_curve(labels, probabilities)

## 11. Sample Predictions Visualization


In [ ]:
def visualize_predictions(num_correct=6, num_incorrect=6):
    """
    Visualize correct and incorrect predictions
    """
    correct_indices = np.where(predictions == labels)[0]
    incorrect_indices = np.where(predictions != labels)[0]
    
    class_names = ['Occupied', 'Vacant']
    
    # Correct predictions
    if len(correct_indices) > 0:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.ravel()
        
        sample_indices = np.random.choice(correct_indices, min(num_correct, len(correct_indices)), replace=False)
        
        for i, idx in enumerate(sample_indices):
            img = cv2.imread(image_paths[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            confidence = probabilities[idx][predictions[idx]]
            
            axes[i].imshow(img)
            axes[i].set_title(
                f"✓ {class_names[predictions[idx]]}\nConfidence: {confidence:.2%}",
                fontsize=11,
                color='green'
            )
            axes[i].axis('off')
        
        plt.suptitle('Correct Predictions', fontsize=16, y=1.00)
        plt.tight_layout()
        plt.savefig(os.path.join(config.RESULTS_DIR, 'correct_predictions.png'), dpi=300, bbox_inches='tight')
        plt.show()
    
    # Incorrect predictions
    if len(incorrect_indices) > 0:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.ravel()
        
        sample_indices = np.random.choice(incorrect_indices, min(num_incorrect, len(incorrect_indices)), replace=False)
        
        for i, idx in enumerate(sample_indices):
            img = cv2.imread(image_paths[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            confidence = probabilities[idx][predictions[idx]]
            
            axes[i].imshow(img)
            axes[i].set_title(
                f"True: {class_names[labels[idx]]}\n"
                f"Pred: {class_names[predictions[idx]]} ({confidence:.2%})",
                fontsize=11,
                color='red'
            )
            axes[i].axis('off')
        
        plt.suptitle('Incorrect Predictions', fontsize=16, y=1.00)
        plt.tight_layout()
        plt.savefig(os.path.join(config.RESULTS_DIR, 'incorrect_predictions.png'), dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("\n🎉 Perfect predictions! No errors found.")

visualize_predictions()

## 12. Comprehensive Results Summary


In [ ]:
# Create comprehensive results summary
results_summary = {
    'Model': config.MODEL_NAME,
    'Test Samples': len(labels),
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'ROC AUC': roc_auc,
    'Sensitivity': sensitivity,
    'Specificity': specificity,
    'Inference Time (ms)': avg_time * 1000,
    'Throughput (FPS)': fps,
    'Correct Predictions': np.sum(predictions == labels),
    'Incorrect Predictions': np.sum(predictions != labels),
    'True Positives': tp,
    'True Negatives': tn,
    'False Positives': fp,
    'False Negatives': fn
}

# Save as JSON
with open(os.path.join(config.RESULTS_DIR, 'test_summary.json'), 'w') as f:
    json.dump(results_summary, f, indent=4)

# Display as DataFrame
summary_df = pd.DataFrame([results_summary]).T
summary_df.columns = ['Value']
print("\n" + "="*60)
print("TEST RESULTS SUMMARY")
print("="*60)
print(summary_df.to_string())
print("="*60)

## 13. Save Detailed Results


In [ ]:
# Create detailed results DataFrame
results_df = pd.DataFrame({
    'image_path': image_paths,
    'true_label': labels,
    'true_label_name': [test_dataset.classes[l] for l in labels],
    'predicted_label': predictions,
    'predicted_label_name': [test_dataset.classes[p] for p in predictions],
    'correct': predictions == labels,
    'confidence': [probabilities[i][predictions[i]] for i in range(len(predictions))],
    'prob_occupied': probabilities[:, 0],
    'prob_vacant': probabilities[:, 1]
})

# Save detailed results
results_csv_path = os.path.join(config.RESULTS_DIR, 'test_results_detailed.csv')
results_df.to_csv(results_csv_path, index=False)
print(f"\n✓ Detailed results saved to {results_csv_path}")

# Save classification report
report = classification_report(
    labels, 
    predictions, 
    target_names=['Occupied', 'Vacant'],
    digits=4
)
with open(os.path.join(config.RESULTS_DIR, 'classification_report.txt'), 'w') as f:
    f.write("PARKING LOT OCCUPANCY ESTIMATION - TEST RESULTS\n")
    f.write("="*60 + "\n\n")
    f.write(f"Model: {config.MODEL_NAME}\n")
    f.write(f"Test Samples: {len(labels)}\n")
    f.write(f"Accuracy: {accuracy:.4f}\n\n")
    f.write(report)
    f.write("\n" + "="*60 + "\n")
    f.write(f"Inference Time: {avg_time*1000:.2f} ms/image\n")
    f.write(f"Throughput: {fps:.2f} FPS\n")

print(f"✓ Classification report saved to classification_report.txt")

# Save metrics to CSV
metrics_df = pd.DataFrame({
    'Metric': list(results_summary.keys()),
    'Value': list(results_summary.values())
})
metrics_df.to_csv(os.path.join(config.RESULTS_DIR, 'test_metrics.csv'), index=False)
print(f"✓ Metrics saved to test_metrics.csv")

print("\n" + "="*60)
print("ALL RESULTS SAVED SUCCESSFULLY!")
print("="*60)
print(f"\nResults directory: {config.RESULTS_DIR}")
print("\nGenerated files:")
print("  • test_results_detailed.csv")
print("  • test_metrics.csv")
print("  • test_summary.json")
print("  • classification_report.txt")
print("  • confusion_matrix.png")
print("  • roc_curve.png")
print("  • correct_predictions.png")
print("  • incorrect_predictions.png")
print("="*60)

## 14. Final Summary


In [ ]:
print("\n\n")
print("╔" + "="*58 + "╗")
print("║" + " "*15 + "FINAL TEST SUMMARY" + " "*25 + "║")
print("╠" + "="*58 + "╣")
print(f"║  Model: {config.MODEL_NAME:<47} ║")
print(f"║  Test Samples: {len(labels):<42} ║")
print("╠" + "="*58 + "╣")
print(f"║  🎯 Accuracy:     {accuracy:.4f} ({accuracy*100:>6.2f}%)" + " "*18 + "║")
print(f"║  📊 Precision:    {precision:.4f}" + " "*35 + "║")
print(f"║  📈 Recall:       {recall:.4f}" + " "*35 + "║")
print(f"║  ⚡ F1-Score:     {f1:.4f}" + " "*35 + "║")
print(f"║  📉 ROC AUC:      {roc_auc:.4f}" + " "*35 + "║")
print("╠" + "="*58 + "╣")
print(f"║  ⏱️  Inference:    {avg_time*1000:>6.2f} ms/image" + " "*24 + "║")
print(f"║  🚀 Throughput:   {fps:>6.2f} FPS" + " "*30 + "║")
print("╠" + "="*58 + "╣")
print(f"║  ✓ Correct:      {np.sum(predictions == labels):<6} ({np.sum(predictions == labels)/len(labels)*100:>5.2f}%)" + " "*22 + "║")
print(f"║  ✗ Incorrect:    {np.sum(predictions != labels):<6} ({np.sum(predictions != labels)/len(labels)*100:>5.2f}%)" + " "*22 + "║")
print("╚" + "="*58 + "╝")

print("\n🎉 Testing complete! All results have been saved.")
print(f"📁 Check {config.RESULTS_DIR} for detailed outputs.\n")